In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.2'
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
home_sales_df


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [ ]:
home_sales_df.createOrReplaceTempView("home_sales")
spark.sql("SELECT * FROM home_sales").show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) year, ROUND(AVG(price),2) avg_price
FROM home_sales
GROUP BY year
ORDER BY year
""").show(truncate=False)


+----+---------+
|year|avg_price|
+----+---------+
|2019|315782.18|
|2020|315949.81|
|2021|313218.09|
|2022|314421.54|
+----+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT date_built build_year, ROUND(AVG(price),2) avg_price
FROM home_sales
WHERE bedrooms=3 AND bathrooms=3
GROUP BY date_built
ORDER BY date_built
""").show(truncate=False)


+----------+---------+
|build_year|avg_price|
+----------+---------+
|2010      |292859.62|
|2011      |291117.47|
|2012      |293683.19|
|2013      |295962.27|
|2014      |290852.27|
|2015      |288770.3 |
|2016      |290555.07|
|2017      |292676.79|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT ROUND(AVG(price),2) avg_price
FROM home_sales
WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living>=2000
""").show(truncate=False)


+---------+
|avg_price|
+---------+
|292538.42|
+---------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(PRICE),2) avg_price
FROM home_sales
WHERE price>=350000
GROUP BY view
ORDER BY avg_price
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|33  |393973.32|
|37  |394964.36|
|6   |395655.38|
|42  |396964.5 |
|2   |397389.25|
|29  |397771.65|
|30  |397862.0 |
|38  |398418.37|
|47  |398447.5 |
|17  |398474.49|
|14  |398570.03|
|8   |398592.71|
|3   |398867.6 |
|13  |398917.98|
|19  |398953.17|
|39  |399234.22|
|18  |399332.91|
|32  |399367.72|
|20  |399522.81|
|27  |399537.66|
+----+---------+
only showing top 20 rows

--- 0.4115262031555176 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(PRICE),2)
FROM home_sales
WHERE price>=350000
GROUP BY view
ORDER BY AVG(price)
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(PRICE), 2)|
+----+--------------------+
|33  |393973.32           |
|37  |394964.36           |
|6   |395655.38           |
|42  |396964.5            |
|2   |397389.25           |
|29  |397771.65           |
|30  |397862.0            |
|38  |398418.37           |
|47  |398447.5            |
|17  |398474.49           |
|14  |398570.03           |
|8   |398592.71           |
|3   |398867.6            |
|13  |398917.98           |
|19  |398953.17           |
|39  |399234.22           |
|18  |399332.91           |
|32  |399367.72           |
|20  |399522.81           |
|27  |399537.66           |
+----+--------------------+
only showing top 20 rows

--- 0.4688105583190918 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
p_home_sales_p=spark.read.parquet('home_sales_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_home_sales_p.createOrReplaceTempView('home_sales_parquet')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(PRICE),2)
FROM home_sales_parquet
WHERE price>=350000
GROUP BY view
ORDER BY AVG(price)
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(PRICE), 2)|
+----+--------------------+
|33  |393973.32           |
|37  |394964.36           |
|6   |395655.38           |
|42  |396964.5            |
|2   |397389.25           |
|29  |397771.65           |
|30  |397862.0            |
|38  |398418.37           |
|47  |398447.5            |
|17  |398474.49           |
|14  |398570.03           |
|8   |398592.71           |
|3   |398867.6            |
|13  |398917.98           |
|19  |398953.17           |
|39  |399234.22           |
|18  |399332.91           |
|32  |399367.72           |
|20  |399522.81           |
|27  |399537.66           |
+----+--------------------+
only showing top 20 rows

--- 0.42968034744262695 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False